Observações Iniciais:

- Na tabela de Cancer tem algumas faixas etarias de determinados estados que não tem valor (estão como 0 na tabela)
- Alguns municipios não possuem dados completos do clima para todos os anos

Transformações Necessarias Observadas:

- Clima
    - Localidade: Tem apenas o nome da cidade e coordenada, terá que preencher o Estado, Pais e Região
    - Data: Tem apenas o ano e o dia, terá que preencher Decada, Mês

- Cancer
    - Localidade: Tem apenas o nome do Estado, terá que preencher Região e Pais. 

In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

# Python 3.11
# Java 11
# PySpark == 3.3.2

In [2]:
spark = SparkSession.builder \
    .appName("OLAP - P2") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-bundle:1.11.1026") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()
    
print(spark.version)

:: loading settings :: url = jar:file:/home/rodrigo/.local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/rodrigo/.ivy2/cache
The jars for the packages stored in: /home/rodrigo/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9f3a4c1c-95ef-4664-9b88-47f5a8fe21dc;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 395ms :: artifacts dl 7ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.2 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

25/06/18 14:30:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


3.3.2


## Extract

In [3]:
df_cancer = spark.read.csv("s3a://datalake/cancer-1.csv", header=True, inferSchema=True)
df_cancer.show()

25/06/18 14:30:17 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/06/18 14:30:17 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
25/06/18 14:30:17 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.
25/06/18 14:30:17 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.


+----------+------------+-----------+-------------+------+---------+------+-----------+--------+--------------------+---------+-----------+----+--------------------+--------------------+--------------------+
|measure_id|measure_name|location_id|location_name|sex_id| sex_name|age_id|   age_name|cause_id|          cause_name|metric_id|metric_name|year|                 val|               upper|               lower|
+----------+------------+-----------+-------------+------+---------+------+-----------+--------+--------------------+---------+-----------+----+--------------------+--------------------+--------------------+
|         1|      Óbitos|       4750|         Acre|     1|Masculino|     8|15 -19 anos|     459|Melanoma maligno ...|        1|     Número|2001|0.008899466563112322|0.010069127561452958|0.007914220987606399|
|         1|      Óbitos|       4750|         Acre|     2| Feminino|     8|15 -19 anos|     459|Melanoma maligno ...|        1|     Número|2001|0.009592002982904217|0.0

In [4]:
df_climate = spark.read.csv("s3a://datalake/climate.csv", header=True, inferSchema=True)
df_climate.show()

+---------------+--------+---------+----+---+-----+-------+-------+-------------------+--------------+--------------+-----------+
|         cidade|latitude|longitude|YEAR|DOY|  T2M|T2M_MAX|T2M_MIN|ALLSKY_SFC_UV_INDEX|ALLSKY_SFC_UVA|ALLSKY_SFC_UVB|PRECTOTCORR|
+---------------+--------+---------+----+---+-----+-------+-------+-------------------+--------------+--------------+-----------+
|Abadia de Goiás|-16.7573| -49.4412|2001|  1|23.12|  26.25|  20.85|               2.29|           1.2|          0.04|      10.58|
|Abadia de Goiás|-16.7573| -49.4412|2001|  2|21.86|  24.46|  19.76|               1.95|          1.05|          0.03|        4.7|
|Abadia de Goiás|-16.7573| -49.4412|2001|  3|21.72|  25.39|  18.91|               2.27|           1.2|          0.04|        4.3|
|Abadia de Goiás|-16.7573| -49.4412|2001|  4|23.14|  27.84|   18.3|               1.81|          0.95|          0.03|       1.72|
|Abadia de Goiás|-16.7573| -49.4412|2001|  5|23.18|  29.38|  17.65|               3.03|   